# Natural gradients

This notebook shows some basic usage of the natural gradient optimizer, both on its own and in combination with Adam optimizer.

In [1]:
import warnings
import numpy as np
import gpflow
import tensorflow as tf

from gpflow.ci_utils import ci_niter, ci_range
from gpflow.models import VGP, GPR, SGPR, SVGP
from gpflow.optimizers import NaturalGradient
from gpflow.optimizers.natgrad import XiSqrtMeanVar

%matplotlib inline
%precision 4

np.random.seed(0)
tf.random.set_seed(0)

N, D = 100, 2
batch_size = 50

# inducing points
M = 10

x = np.random.uniform(size=(N, D))
y = np.sin(10 * x)

data = (x, y)
inducing_variable = tf.random.uniform((M, D))
adam_learning_rate = 0.01
iterations = ci_niter(5)

### VGP is a GPR

The following section demonstrates how natural gradients can turn VGP into GPR *in a single step, if the likelihood is Gaussian*.

Let's start by first creating a standard GPR model with Gaussian likelihood:

In [2]:
gpr = GPR(data, kernel=gpflow.kernels.Matern52())

The likelihood of the exact GP model is:

In [3]:
gpr.log_likelihood().numpy()

-231.08993573018935

Now we will create an approximate model which approximates the true posterior via a variational Gaussian distribution.<br>We initialize the distribution to be zero mean and unit variance.

In [4]:
vgp = VGP(data, kernel=gpflow.kernels.Matern52(), likelihood=gpflow.likelihoods.Gaussian())

The likelihood of the approximate GP model is:

In [5]:
vgp.log_likelihood().numpy()

-328.84293853435463

Obviously, our initial guess for the variational distribution is not correct, which results in a lower bound to the likelihood of the exact GPR model. We can optimize the variational parameters in order to get a tighter bound. 

In fact, we only need to take **one step** in the natural gradient direction to recover the exact posterior:

In [6]:
natgrad_opt = NaturalGradient(gamma=1.0)
variational_params = [(vgp.q_mu, vgp.q_sqrt)]
natgrad_opt.minimize(lambda: - vgp.log_marginal_likelihood(), var_list=variational_params)

The likelihood of the approximate GP model after a single NatGrad step:

In [7]:
vgp.log_likelihood().numpy()

-231.08999824749554

### Optimize both variational parameters and kernel hyperparameters together

In the Gaussian likelihood case we can iterate between an Adam update for the hyperparameters and a NatGrad update for the variational parameters. That way, we achieve optimization of hyperparameters as if the model were a GPR.

The trick is to forbid Adam from updating the variational parameters by setting them to not trainable.

In [8]:
# Stop Adam from optimizing the variational parameters
vgp.q_mu.trainable = False
vgp.q_sqrt.trainable = False

adam_opt_for_vgp = tf.optimizers.Adam(adam_learning_rate)
adam_opt_for_gpr = tf.optimizers.Adam(adam_learning_rate)

In [9]:
for i in range(iterations):
    adam_opt_for_gpr.minimize(
        lambda: - gpr.log_marginal_likelihood(), 
        var_list=gpr.trainable_variables)
    likelihood = gpr.log_likelihood()
    tf.print(f'GPR with Adam: iteration {i + 1} likelihood {likelihood:.04f}')

GPR with Adam: iteration 1 likelihood -230.6706
GPR with Adam: iteration 2 likelihood -230.2508
GPR with Adam: iteration 3 likelihood -229.8303
GPR with Adam: iteration 4 likelihood -229.4093
GPR with Adam: iteration 5 likelihood -228.9876


In [10]:
for i in range(iterations):
    adam_opt_for_vgp.minimize(
        lambda: - vgp.log_marginal_likelihood(), 
        var_list=vgp.trainable_variables)
    natgrad_opt.minimize(
        lambda: - vgp.log_marginal_likelihood(), 
        var_list=variational_params)
    likelihood = vgp.log_likelihood()
    tf.print(f'VGP with NaturalGradient and Adam: iteration {i + 1} likelihood {likelihood:.04f}')

VGP with NaturalGradient and Adam: iteration 1 likelihood -230.6707
VGP with NaturalGradient and Adam: iteration 2 likelihood -230.2508
VGP with NaturalGradient and Adam: iteration 3 likelihood -229.8304
VGP with NaturalGradient and Adam: iteration 4 likelihood -229.4093
VGP with NaturalGradient and Adam: iteration 5 likelihood -228.9877


Compare GPR and VGP lengthscales after optimization:

In [11]:
print(f'GPR lengthscales = {gpr.kernel.lengthscale.numpy():.04f}')
print(f'VGP lengthscales = {vgp.kernel.lengthscale.numpy():.04f}')

GPR lengthscales = 0.9686
VGP lengthscales = 0.9686


### Natural gradients also work for the sparse model
Similarly, natural gradients turn SVGP into SGPR in the Gaussian likelihood case. <br>
We can again combine natural gradients with Adam to update both variational parameters and hyperparameters too.<br>
Here we'll just do a single natural step demonstration.

In [12]:
svgp = SVGP(kernel=gpflow.kernels.Matern52(), likelihood=gpflow.likelihoods.Gaussian(), inducing_variable=inducing_variable)
sgpr = SGPR(data, kernel=gpflow.kernels.Matern52(), inducing_variable=inducing_variable)

for model in svgp, sgpr:
    model.likelihood.variance.assign(0.1)

Analytically optimal sparse model likelihood:

In [13]:
sgpr.log_likelihood().numpy()

-301.3217357213807

SVGP likelihood before natural gradient step:

In [14]:
svgp.log_likelihood(data).numpy()

-1404.0804961509486

In [15]:
variational_params = [(svgp.q_mu, svgp.q_sqrt)]

def svgp_loss_cb():
    return - svgp.log_marginal_likelihood(data)

natgrad_opt = NaturalGradient(gamma=1.0)
natgrad_opt.minimize(svgp_loss_cb, var_list=variational_params)

SVGP likelihood after a single natural gradient step:

In [16]:
svgp.log_likelihood(data).numpy()

-315.41174669482126

### Minibatches
A crucial property of the natural gradient method is that it still works with minibatches.
In practice though, we need to use a smaller gamma.

In [18]:
natgrad_opt = NaturalGradient(gamma=0.1)

data_minibatch = tf.data.Dataset.from_tensor_slices(data).prefetch(N).repeat().shuffle(N).batch(batch_size)
data_minibatch_it = iter(data_minibatch)

def svgp_stochastic_loss_cb() -> tf.Tensor:
    batch = next(data_minibatch_it)
    return - svgp.log_marginal_likelihood(batch)

for _ in range(ci_niter(100)):
    natgrad_opt.minimize(svgp_stochastic_loss_cb, var_list=variational_params)

Minibatch SVGP likelihood after NatGrad optimization:

In [19]:
np.average([svgp.log_likelihood(next(data_minibatch_it)) for _ in ci_range(100)])

-166.7657946763955

### Comparison with ordinary gradients in the conjugate case

##### (Take home message: natural gradients are always better)

Compared to SVGP with ordinary gradients with minibatches, the natural gradient optimizer is much faster in the Gaussian case. 

Here we'll do hyperparameter learning together with optimization of the variational parameters, comparing the interleaved natural gradient approach and the one using ordinary gradients for the hyperparameters and variational parameters jointly.

**NOTE:** Again we need to compromise for smaller gamma value, which we'll keep *fixed* during the optimization.

In [20]:
svgp_ordinary = SVGP(kernel=gpflow.kernels.Matern52(), likelihood=gpflow.likelihoods.Gaussian(), inducing_variable=inducing_variable)
svgp_natgrad = SVGP(kernel=gpflow.kernels.Matern52(), likelihood=gpflow.likelihoods.Gaussian(), inducing_variable=inducing_variable)

# ordinary gradients with Adam for SVGP
ordinary_adam_opt = tf.optimizers.Adam(adam_learning_rate)

# NatGrads and Adam for SVGP
# Stop Adam from optimizing the variational parameters
svgp_natgrad.q_mu.trainable = False
svgp_natgrad.q_sqrt.trainable = False

# Create the optimize_tensors for SVGP
natgrad_adam_opt = tf.optimizers.Adam(adam_learning_rate)

natgrad_opt = NaturalGradient(gamma=0.1)
variational_params = [(svgp_natgrad.q_mu, svgp_natgrad.q_sqrt)]

Let's optimize the models:

In [20]:
data_minibatch = tf.data.Dataset.from_tensor_slices(data).prefetch(N).repeat().shuffle(N).batch(batch_size)
data_minibatch_it = iter(data_minibatch)


def svgp_ordinary_loss_cb() -> tf.Tensor:
    batch = next(data_minibatch_it)
    return - svgp_ordinary.log_marginal_likelihood(batch)


def svgp_natgrad_loss_cb() -> tf.Tensor:
    batch = next(data_minibatch_it)
    return - svgp_natgrad.log_marginal_likelihood(batch)


for _ in range(ci_niter(100)):
    ordinary_adam_opt.minimize(svgp_ordinary_loss_cb, var_list=svgp_ordinary.trainable_variables)


for _ in range(ci_niter(100)):
    natgrad_adam_opt.minimize(svgp_natgrad_loss_cb, var_list=svgp_natgrad.trainable_variables)
    natgrad_opt.minimize(svgp_natgrad_loss_cb, var_list=variational_params)

SVGP likelihood after ordinary `Adam` optimization:

In [21]:
np.average([svgp_ordinary.log_likelihood(next(data_minibatch_it)) for _ in ci_range(100)])

-104.49762723906062

SVGP likelihood after `NaturalGradient` and `Adam` optimization:

In [22]:
np.average([svgp_natgrad.log_likelihood(next(data_minibatch_it)) for _ in ci_range(100)])

-102.54594520791255

### Comparison with ordinary gradients in the non-conjugate case
#### Binary classification

##### (Take home message: natural gradients are usually better)

We can use natural gradients even when the likelihood isn't Gaussian. It isn't guaranteed to be better, but it usually is better in practical situations.

In [23]:
y_binary = np.random.choice([1., -1], size=x.shape)
vgp_data = (x, y_binary)

vgp_bernoulli = VGP(vgp_data, kernel=gpflow.kernels.Matern52(), likelihood=gpflow.likelihoods.Bernoulli())
vgp_bernoulli_natgrad = VGP(vgp_data, kernel=gpflow.kernels.Matern52(), likelihood=gpflow.likelihoods.Bernoulli())

# ordinary gradients with Adam for VGP with Bernoulli likelihood
adam_opt = tf.optimizers.Adam(adam_learning_rate)

# NatGrads and Adam for VGP with Bernoulli likelihood
# Stop Adam from optimizing the variational parameters
vgp_bernoulli_natgrad.q_mu.trainable = False
vgp_bernoulli_natgrad.q_sqrt.trainable = False

# Create the optimize_tensors for VGP with natural gradients
natgrad_adam_opt = tf.optimizers.Adam(adam_learning_rate)
natgrad_opt = NaturalGradient(gamma=0.1)
variational_params = [(vgp_bernoulli_natgrad.q_mu, vgp_bernoulli_natgrad.q_sqrt)]

In [24]:
# Optimize vgp_bernoulli
for _ in range(ci_niter(100)):
    adam_opt.minimize(
        lambda: - vgp_bernoulli.log_marginal_likelihood(),
        var_list=vgp_bernoulli.trainable_variables)

# Optimize vgp_bernoulli_natgrad
for _ in range(ci_niter(100)):
    adam_opt.minimize(
        lambda: - vgp_bernoulli_natgrad.log_marginal_likelihood(),  
        var_list=vgp_bernoulli_natgrad.trainable_variables)
    natgrad_opt.minimize(
            lambda: - vgp_bernoulli_natgrad.log_marginal_likelihood(),
            var_list=variational_params)

VGP likelihood after ordinary `Adam` optimization:

In [25]:
vgp_bernoulli.log_likelihood().numpy()

-144.21400720445877

VGP likelihood after `NaturalGradient` + `Adam` optimization:

In [26]:
vgp_bernoulli_natgrad.log_likelihood().numpy()

-142.83799599076423

We can also choose to run natural gradients in another parameterization.<br>
The sensible choice is the model parameters (q_mu, q_sqrt), which is already in GPflow.

In [27]:
vgp_bernoulli_natgrads_xi = VGP(vgp_data, kernel=gpflow.kernels.Matern52(), likelihood=gpflow.likelihoods.Bernoulli())

# Stop Adam from optimizing the variational parameters
vgp_bernoulli_natgrads_xi.q_mu.trainable = False
vgp_bernoulli_natgrads_xi.q_sqrt.trainable = False

# Create the optimize_tensors for VGP with Bernoulli likelihood
adam_opt = tf.optimizers.Adam(adam_learning_rate)
natgrad_opt = NaturalGradient(gamma=0.01)

variational_params = [(vgp_bernoulli_natgrads_xi.q_mu, vgp_bernoulli_natgrads_xi.q_sqrt, XiSqrtMeanVar())]

In [28]:
# Optimize vgp_bernoulli_natgrads_xi
for _ in range(ci_niter(100)):
    adam_opt.minimize(
        lambda: - vgp_bernoulli_natgrads_xi.log_marginal_likelihood(),                
        var_list=vgp_bernoulli_natgrads_xi.trainable_variables)

    natgrad_opt.minimize(
        lambda: - vgp_bernoulli_natgrads_xi.log_marginal_likelihood(),
        var_list=variational_params)

W1021 21:53:24.205371 4725601728 backprop.py:980] Calling GradientTape.gradient on a persistent tape inside its context is significantly less efficient than calling it outside the context (it causes the gradient ops to be recorded on the tape, leading to increased CPU and memory usage). Only call GradientTape.gradient inside the context if you actually want to trace the gradient in order to compute higher order derivatives.


VGP likelihood after `NaturalGradient` with `XiSqrtMeanVar` + `Adam` optimization:

In [29]:
vgp_bernoulli_natgrads_xi.log_likelihood().numpy()

-143.09890480317517

With sufficiently small steps, it shouldn't make a difference which transform is used, but for large 
steps this can make a difference in practice.